In [ ]:
library(ggplot2)
library(shiny)
draws <- 1000 # number of steps to sample in one trial
trials <- 5000 # number of trials(more trials make the graph clearer)
ratio_multi <- matrix(0, trials,1) #create a matrix to record the ratio of black balls in the urn over time

In [ ]:
ui <- fluidPage(
  # Application title
  titlePanel("Polya Urn Model"),
  h4("Based on the Polya urn model, this program will create histograms of observed ratios of black balls in the urn after 
1000 draws(steps) for 5000 trials with different initial conditions (a, b), compared to the density function of 
the Beta(a,b) distribution. (a: the initial number of black balls in the urn; b: the initial number of red balls)"),
  br(),
  h4("Change the initial numbers of black balls(a) and red balls(b) to any number you would like to explore.
  Since the total number of iterations is huge, so after clicking the Run button, 
     the histogram will take about 40-60 seconds to show up."),
  br(),
  h4("As shown by the graph, after a large number of trials, the proportion of black balls in the urn 
     converges in distribution to the Beta distribution with parameters(a, b)"),
  br(),
     # Sidebar with a slider input for number of bins
  sidebarLayout(
    sidebarPanel(
      numericInput("a",
                   "Initial number of black balls (a):",
                   value=1),
      
      numericInput("b",
                   "Initial number of red balls (b):",
                   value = 1),
      
      actionButton("start", "Run")
    ),
    
    # Show a plot of the generated distribution
    mainPanel(
      plotOutput("polya_urn_multi")
    )
  )
)
    
    # Define server logic required to draw a histogram
    server <- function(input, output) {
      output$polya_urn_multi <- renderPlot({
        black <- input$a
        red<- input$b
        polya_urn_multi <- function(a, b) {
          for (i in 1:trials){
          black_curr <- a #current number of black balls
          red_curr <- b #current number of red balls
          for (n in 1:draws){
            # the prob of drawing a black ball is (current # of black balls)/(initial # of black+red balls + current # of steps)
            # the prob of drawing a red ball is (current # of red balls)/(initial # of black+red balls + current # of steps)
            # Given those probabilities, sample a ball from the urn, 
            # here we use 1 to represent a balck ball and 0 to represent a red ball
            drawn<-sample(c(1,0), size=1, prob=c(black_curr, red_curr)/(a + b + n))
            if (drawn == 1) {
              black_curr <- black_curr + 1
            } else {
              red_curr <- red_curr + 1
            }
          }
          # in the ith trial, add the ratio of black balls in the urn after n steps
          ratio_multi[i]<-black_curr/(a + b + n)
        }
        # the proportion of black balls in the urn converges in distribution 
        # to the Beta distribution with parameters the same as initial # of red balls
        # and initial # of black balls when n grows.
        # As shown by the graph
        sort_ratio <- sort(ratio_multi)
        beta_pdf <- dbeta(sort_ratio, a, b)
        df <- data.frame(ratio_multi,beta_pdf)
        beta_para <- paste("BETA(",a,",",b,")")
        num_bins <- 20
        urn_plot <- ggplot(df, aes(ratio_multi)) +                  
          geom_histogram(aes(x = ratio_multi), fill="lightblue", bins = num_bins, boundary=0, color = "white") + 
          scale_x_continuous(breaks = seq(0, 1, by=0.2), limits=c(0,1)) +
          labs(title = beta_para,
               x = "Ratio of Black Balls in the Urn",
               y = "") + 
          theme_bw(base_size = 16) +
          theme(plot.title = element_text(hjust = 0.5, face = "bold"), 
                axis.text.y = element_blank(), 
                plot.margin = unit(c(1, 1, 1, 2), "cm")) + 
          geom_line(aes(x = sort_ratio, y= (trials/num_bins) * beta_pdf), colour="red")
        print(urn_plot)
        }
        polya_urn_multi(black, red)
      })
    }
    # Run the application 
    shinyApp(ui = ui, server = server)